### Modelos de Recuperaciòn de Informaciòn (y evaluaciòn)

Utilizando la colecci´on provista por el equipo docente1
, cuya estructura es la siguiente:
    
vocabulary.txt → [id termino, idf, t´ermino]

documentVectors.txt → [id doc, lista(id terminos)]

queries.txt → [id query, lista(id terminos)]

relevants.txt → [id query, listarelevantes (id doc)]

informationNeeds.txt → [id in, texto libre]


a) Calcule los conjuntos de respuestas usando el modelo booleano y el modelo vectorial (asuma en todos
los casos T F = 1).

In [136]:
from os import listdir
from os.path import isdir
import math
from modulos.tokenizer import *

In [137]:
def calc_idf(corpus_count,doc_freq):
    if(doc_freq !=0):
        return math.log(corpus_count/doc_freq,2)
    return 0

In [138]:
def indexer(dirname):
        files = listdir(dirname)
        vocabulary={}
        document_vector={}
        id_voc=0
        vocavulary_result=[]
        docs_count=len(files)
        for file in files:
            lines = open(dirname+'/'+file,'r',errors = 'ignore').readlines()
            docu_voc=[]
            tokens=tokenizar(lines)
            #docs_count+=1
            for token in tokens:
                if token not in vocabulary:
                    vocabulary[token]=(id_voc,1)
                    docu_voc.append(id_voc)
                    id_voc =id_voc+1
                    #print(token,docu_voc)
                    
                    
                else:
                    id,doc_freq = vocabulary[token]
                    
                    #print(id,docu_voc)
                    if id not in docu_voc:
                       
                        doc_freq = doc_freq+1
                        docu_voc.append(id)
                        
                    vocabulary[token]=(id,doc_freq)
                    
            document_vector[file]=docu_voc
        #print(document_vector)
        #print(vocabulary)
        for termn in vocabulary.items():
            #print(termn[1][1],docs_count)
            vocavulary_result.append([termn[1][0],calc_idf(docs_count,termn[1][1]),termn[0]])
        #print (vocavulary_result)
        return (vocavulary_result,document_vector)

In [139]:
vocabulary,documents=  indexer('/home/agu/Unlu/IR/tp2_analisis_texto/data')
print(vocabulary)
print(documents)

[[0, 0.5849625007211562, 'bye'], [1, 1.5849625007211563, 'hose'], [2, 1.5849625007211563, 'house'], [3, 0.0, 'dog'], [4, 1.5849625007211563, 'http'], [5, 1.5849625007211563, 'url'], [6, 1.5849625007211563, 'com'], [7, 1.5849625007211563, 'ar'], [8, 1.5849625007211563, 'agustin'], [9, 1.5849625007211563, 'rodriguez'], [10, 0.5849625007211562, 'agu'], [11, 1.5849625007211563, 'gmail'], [12, 1.5849625007211563, 'www'], [13, 1.5849625007211563, 'https'], [14, 1.5849625007211563, 'pepe'], [15, 1.5849625007211563, 'run'], [16, 0.5849625007211562, 'hello'], [17, 1.5849625007211563, 'sleep'], [18, 0.5849625007211562, 'program'], [19, 0.5849625007211562, 'programer'], [20, 1.5849625007211563, 'lic'], [21, 1.5849625007211563, 'programs'], [22, 1.5849625007211563, 'programing']]
{'ir.txt': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'file1.txt': [15, 16, 3, 17, 18, 19, 20, 10], 'archivo1.txt': [16, 0, 3, 18, 21, 19, 22]}


In [140]:
import numpy as np
from operator import itemgetter

In [141]:
#voc = (list(get_vocabulary("ejemploRibeiro/vocabulary.txt", True)))
#doc = (get_document_vector("ejemploRibeiro/documentVector.txt", True))
voc = vocabulary
vocabulary_list=list(map(itemgetter(2), voc))
#doc = documents
#doc_bool= to_boolean(doc,voc)
print (voc)
print(documents)

[[0, 0.5849625007211562, 'bye'], [1, 1.5849625007211563, 'hose'], [2, 1.5849625007211563, 'house'], [3, 0.0, 'dog'], [4, 1.5849625007211563, 'http'], [5, 1.5849625007211563, 'url'], [6, 1.5849625007211563, 'com'], [7, 1.5849625007211563, 'ar'], [8, 1.5849625007211563, 'agustin'], [9, 1.5849625007211563, 'rodriguez'], [10, 0.5849625007211562, 'agu'], [11, 1.5849625007211563, 'gmail'], [12, 1.5849625007211563, 'www'], [13, 1.5849625007211563, 'https'], [14, 1.5849625007211563, 'pepe'], [15, 1.5849625007211563, 'run'], [16, 0.5849625007211562, 'hello'], [17, 1.5849625007211563, 'sleep'], [18, 0.5849625007211562, 'program'], [19, 0.5849625007211562, 'programer'], [20, 1.5849625007211563, 'lic'], [21, 1.5849625007211563, 'programs'], [22, 1.5849625007211563, 'programing']]
{'ir.txt': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'file1.txt': [15, 16, 3, 17, 18, 19, 20, 10], 'archivo1.txt': [16, 0, 3, 18, 21, 19, 22]}


In [142]:
def get_query(query):
    
    query_result=[]
    querys = query.split()
    
    for term  in querys:
        if term in vocabulary_list:
            query_result.append(vocabulary_list.index(term))  
    return query_result

In [162]:
query =get_query("hello dog agustin")
print(query)

[16, 3, 8]


In [163]:
result=[]
query_documents=[]
for term in query:


    for document, values in documents.items():

        if term in values:
            if document not in query_documents:   
                query_documents.append(document)

print(query_documents)


['file1.txt', 'archivo1.txt', 'ir.txt']


In [164]:
def get_normal(list_term,vocabulary):
    query_normal=0
    for term in list_term:
        val=(vocabulary[term][1])
        query_normal+=float(val)**2
    query_normal=math.sqrt(query_normal)
    #print(query_normal,"fnc")
    return(query_normal)

In [165]:
result=[]
print(query)
#print(get_normal(query,voc))
query_normal=get_normal(query,voc)

result_q=[]

terms_isin_documents={}

for term in query:
    if voc[term][1] > 0:
        query_document=[]
        #query_document=[]
        #for document in doc_bool:
        for document, values in documents.items():
            #docu, value = doc
            if term in values:
            #if (value[term-1]):
                #a =[ids for ids, _ in query_document]
                #print (a)
                #print(docu[0])
                if (document not in [ids for ids, _ in result_q]):
                    query_document.append((document,voc[term][1]))

            if(term in terms_isin_documents.keys()):
                 #print(terms_isin_documents.keys())
                #print(query_document)
                terms_isin_documents[str(term)].extend(query_document)     
            else:
                if len(query_document)>0:
                    terms_isin_documents[str(term)]=query_document
        result_q.extend(query_document)
#print("query",query_document) 
result.append(("query",result_q))

print("Boolean Result")
for doc,_ in result_q:
    print(doc)
#print(result_q)
query_vector=[]
for key,value in terms_isin_documents.items():
    query_vector.append(float(voc[int(key)][1]))


vector_result=[]
for name,idf in result_q:
    #print(result_q)
    vect_sim =[]
    sim=0
    #value son los documentos que contienen los terminos
    #print(terms_isin_documents.items())
    for key,value in terms_isin_documents.items():
        if (name in [val[0] for val in value]):
            vect_sim.append(float(idf))
        else:
            vect_sim.append(0)
    vector_result.append((name,vect_sim))
print("------------------------")
print("Vectorial Result")
#print(vector_result)
list_response=[]
print(query_normal)
for doc,vec in vector_result:
    #r=np.array(r)
    q=0

    for idf in vec:
        q+=idf**2
    q = math.sqrt(q)
    q = (np.dot(vec,query_vector)) /(query_normal*q)
    list_response.append((doc,q))


(list_response.sort(key=itemgetter(1),reverse=True))

for doc in list_response:
    print(doc)
print("-----------------")
    
    

[16, 3, 8]
Boolean Result
file1.txt
archivo1.txt
ir.txt
------------------------
Vectorial Result
1.6894636000642955
('ir.txt', 0.9381453975456103)
('file1.txt', 0.34624155305796134)
('archivo1.txt', 0.34624155305796134)
-----------------
